# 1. 카메라 내부파라미터 추론

In [1]:
import cv2
import numpy as np
import glob

#test

# 체스보드 패턴의 내부 코너 개수
checkerboard_size = (9, 6)

# 체스보드 패턴의 각 코너에 대한 3D 좌표 설정
objp = np.zeros((checkerboard_size[0]*checkerboard_size[1], 3), np.float32)
objp[:, :2] = np.mgrid[0:checkerboard_size[0], 0:checkerboard_size[1]].T.reshape(-1, 2)

# 3D 점과 2D 점 저장을 위한 배열
objpoints = []  # 실제 3D 좌표
imgpoints = []  # 이미지 2D 좌표

# 체스보드 이미지 파일 읽기
images = glob.glob('imagesforimplicit/*.jpg')

for fname in images:
    img = cv2.imread(fname)
    img = cv2.resize(img, (1920, 1080))
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # 체스보드 코너 찾기
    ret, corners = cv2.findChessboardCorners(gray, checkerboard_size, None)
    
    if ret:
        objpoints.append(objp)
        imgpoints.append(corners)
        
        # 코너를 그려서 확인
        img = cv2.drawChessboardCorners(img, checkerboard_size, corners, ret)
        cv2.imshow('img', img)
        cv2.waitKey(500)

cv2.destroyAllWindows()

# 카메라 캘리브레이션 수행
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

# 결과 출력
print("카메라 행렬 (내부 파라미터):")
print(mtx)
print("\n왜곡 계수:")
print(dist)


카메라 행렬 (내부 파라미터):
[[1.36626365e+03 0.00000000e+00 9.71802057e+02]
 [0.00000000e+00 1.02508832e+03 5.53490799e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]

왜곡 계수:
[[ 0.19233521 -0.3909514   0.00646862 -0.00067985 -0.11904668]]


# 2. 카메라 외부파라미터 추론

In [2]:
# 좌표축 그리기 함수
def draw_axes(img, corners, imgpts, corner_idx = 0):
    # 각 포인트의 좌표를 정수형으로 변환
    corner = tuple(int(x) for x in corners[corner_idx].ravel())
    img = cv2.line(img, corner, tuple(int(x) for x in imgpts[0].ravel()), (255,0,0), 5)
    img = cv2.line(img, corner, tuple(int(x) for x in imgpts[1].ravel()), (0,255,0), 5)
    img = cv2.line(img, corner, tuple(int(x) for x in imgpts[2].ravel()), (0,0,255), 5)
    return img


# 회전 벡터를 변환하는 함수
def transform_rvec_rotation(rvec):
    R, _ = cv2.Rodrigues(rvec)
    R_z_90 = np.array([[0, -1, 0],
                       [1, 0, 0],
                       [0, 0, 1]])
    R_y_180 = np.array([[-1, 0, 0],
    [0, 1, 0],
    [0, 0, -1]])
    
    R_new = R @ R_y_180 @ R_z_90
    rvec_new, _ = cv2.Rodrigues(R_new)
    return rvec_new


In [3]:
# 체커보드의 차원 정의
CHECKERBOARD = (4, 6)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# 3D 점의 세계 좌표 정의
objp = np.zeros((CHECKERBOARD[0] * CHECKERBOARD[1], 3), np.float32)
objp[:, :2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)

# 주어진 디렉터리에 저장된 개별 이미지의 경로 추출
image = './imagesforexplicit/Right.jpg'

# 이미 알고 있는 내부 파라미터 행렬 (예시로 설정)
K1 = np.array([[1.12565025e+03, 0.00000000e+00, 9.60006129e+02],
               [0.00000000e+00, 1.12789841e+03, 5.68305880e+02],
               [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

# 왜곡 계수 (예시로 설정)
dist = np.array([[0.12742061, -0.45943817, 0.00251015, 0.00197737, 1.14483895]])

# 외부 파라미터를 저장할 리스트
Rmat = []
tmat = []

# 축의 길이 정의
axis = np.float32([[3,0,0], [0,3,0], [0,0,3]]).reshape(-1,3)

img = cv2.imread(image)
img = cv2.resize(img, (1920, 1080))

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, None)
if ret:
    corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
    ret, rvec, tvec = cv2.solvePnP(objp, corners2, K1, dist)
    
    # 회전 변환 적용
    rvec_transformed = transform_rvec_rotation(rvec)
    
    Rmat.append(cv2.Rodrigues(rvec_transformed)[0])
    tmat.append(tvec * 200)
    
    imgpts, jac = cv2.projectPoints(axis, rvec_transformed, tvec, K1, dist)
    img = cv2.drawChessboardCorners(img, CHECKERBOARD, corners2, ret)
    img = draw_axes(img, corners2, imgpts, 0)
    cv2.imshow('Image with World Origin', img)
    cv2.imwrite("./output/"+image.split('\\')[-1], img)
    cv2.waitKey(0)
cv2.destroyAllWindows()

print("회전 행렬 R1:")
print(Rmat[0])
print("변환 벡터 t1:")
print(tmat[0])

회전 행렬 R1:
[[-0.13812448  0.99039822 -0.0057441 ]
 [ 0.15067916  0.01528137 -0.9884646 ]
 [-0.9788858  -0.13739668 -0.1513431 ]]
변환 벡터 t1:
[[-728.5138225 ]
 [ 628.73961129]
 [4647.41077983]]


In [4]:
# 3D 점의 세계 좌표 정의
objp = np.zeros((CHECKERBOARD[0] * CHECKERBOARD[1], 3), np.float32)
objp[:, :2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
objp[:, 0] = CHECKERBOARD[0] - 1 - objp[:, 0]  # x 좌표 반전
objp[:, 1] = CHECKERBOARD[1] - 1 - objp[:, 1]  # y 좌표 반전

# 주어진 디렉터리에 저장된 개별 이미지의 경로 추출
image = './imagesforexplicit/Left.jpg'

# 이미 알고 있는 내부 파라미터 행렬 (예시로 설정)
K2 = np.array([[1.12565025e+03, 0.00000000e+00, 9.60006129e+02],
               [0.00000000e+00, 1.12789841e+03, 5.68305880e+02],
               [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

img = cv2.imread(image)
img = cv2.resize(img, (1920, 1080))

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, None)
if ret:
    corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
    ret, rvec, tvec = cv2.solvePnP(objp, corners2, K2, dist)
    
    # 회전 변환 적용
    rvec_transformed = transform_rvec_rotation(rvec)
    
    Rmat.append(cv2.Rodrigues(rvec_transformed)[0])
    tmat.append(tvec * 200)
    
    imgpts, jac = cv2.projectPoints(axis, rvec_transformed, tvec, K2, dist)
    img = cv2.drawChessboardCorners(img, CHECKERBOARD, corners2, ret)
    img = draw_axes(img, corners2, imgpts, 23)
    cv2.imshow('Image with World Origin', img)
    cv2.imwrite("./output/"+image.split('\\')[-1], img)
    cv2.waitKey(0)
cv2.destroyAllWindows()

print("회전 행렬 R2:")
print(Rmat[1])
print("변환 벡터 t2:")
print(tmat[1])

회전 행렬 R2:
[[-0.48808903 -0.87236478 -0.02736389]
 [-0.1178842   0.09695628 -0.98828275]
 [ 0.86479617 -0.4791442  -0.15016132]]
변환 벡터 t2:
[[ 982.56148793]
 [ 918.26381763]
 [3063.3502418 ]]
